In [ ]:
from keras import backend as K
import sys
import math
from tqdm import tqdm
from skimage.io import imread, imshow
from skimage.transform import resize
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from skimage.exposure import equalize_hist
from google.colab.patches import cv2_imshow
import cv2
import numpy as np
from PIL import Image
IMG_HEIGHT = 256
IMG_WIDTH = 208
IMG_CHANNEL = 1 
from google.colab import drive
drive.mount('/content/drive')
!pip install pydicom
import pydicom
e = 2.7182

In [ ]:
def Transform_data_train(X,y):
    args = dict(
        rotation_range = 20,
        horizontal_flip = True,
        vertical_flip = True)
    img_datagen = ImageDataGenerator(**args)
    mask_datagen = ImageDataGenerator(**args)
    X_gen = img_datagen.flow(X, y, batch_size=4)
    # y_gen = mask_datagen.flow(y, batch_size=4)
    # gen = zip(X_gen, y_gen)
    return X_gen

def Transform_data_val(X,y):
    args = dict(
        rotation_range = 20,
        horizontal_flip = True,
        vertical_flip = True)
    img_datagen = ImageDataGenerator(**args)
    mask_datagen = ImageDataGenerator(**args)

    X_gen = img_datagen.flow(X, y, batch_size=1)
    # y_gen = mask_datagen.flow(y, batch_size=1)
    # gen = zip(X_gen, y_gen)
    return X_gen

In [ ]:
import glob, os
Icountour_Train = glob.glob('/content/drive/My Drive/RVData/unrar_files/TrainingSet/*-icontour-manual.txt')
Ocountour_Train = glob.glob('/content/drive/My Drive/RVData/unrar_files/TrainingSet/*-ocontour-manual.txt')
IMG_Train = []
for i in range(0,len(Icountour_Train)):
  IMG_Train.append(Icountour_Train[i][:63]+'.dcm')

X_train = []
y_train = []
for i, filename in enumerate(IMG_Train):
  img = pydicom.dcmread(IMG_Train[i]).pixel_array.astype(int)
  img_shape = img.shape
  masks = np.loadtxt(Ocountour_Train[i]).astype(int)
  mask_img = np.zeros(img_shape, dtype = np.float32)
  cv2.fillPoly(mask_img, pts = [masks], color = 1)
  if img_shape == (216,IMG_HEIGHT):
        img = np.transpose(img)
        mask_img = np.transpose(mask_img)
  img = img[:, 4:212]
  mask_img = mask_img[:,4:212]
  img = np.expand_dims(equalize_hist(image= img), axis = 2)
  X_train.append(img)
  mask_img = np.expand_dims(mask_img, axis = 2)
  y_train.append(mask_img)

In [ ]:
X_val = np.array(X_train[200:], dtype = np.float32)
y_val = np.array(y_train[200:], dtype = np.float32)
X_train = np.array(X_train[:200], dtype = np.float32)
y_train = np.array(y_train[:200], dtype = np.float32)
Train_data = Transform_data_train(X_train, y_train)
Val_data = Transform_data_val(X_val, y_val)
# X_train, y_train = Transform_data_train(X_train, y_train)
# X_val, y_val = Transform_data_val(X_val, y_val)
# print(X_train)
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

In [ ]:
def SmoothDiceBCE_Loss(targets, inputs, eps = 1e-6):
	inputs = K.reshape(inputs, (K.shape(inputs)[0],-1))
	targets = K.reshape(targets, (K.shape(targets)[0],-1))
	BCE = K.mean(K.binary_crossentropy(targets,inputs),axis = 1)
	intersection = K.sum(targets*inputs, axis = 1)
	Dice_Loss = 1 - (2*intersection+eps)/(K.sum(targets, axis = 1) + K.sum(inputs, axis = 1) + eps)
	Final = BCE + Dice_Loss
	return Final

def SmoothDiceBCEInvDice_Loss(targets, inputs, eps = 1e-6):
	inputs = K.reshape(inputs, (K.shape(inputs)[0],-1))
	targets = K.reshape(targets, (K.shape(targets)[0],-1))
	BCE = K.mean(K.binary_crossentropy(targets,inputs),axis = 1)
	intersection = K.sum(targets*inputs, axis = 1)
	Dice_Loss = 1 - (2*intersection+eps)/(K.sum(targets, axis = 1) + K.sum(inputs, axis = 1) + eps)
	Inv_inputs = 1 - inputs
	Inv_targets = 1 - targets
	Inv_intersection = K.sum(Inv_targets*Inv_inputs, axis = 1)
	Inv_Dice_Loss = 1 - (2*Inv_intersection+eps)/(K.sum(Inv_targets, axis = 1) + K.sum(Inv_inputs, axis = 1) + eps)
	Final = BCE+Dice_Loss+Inv_Dice_Loss
	return Final

def Switching_Loss(targets, inputs, l = 0.75, t = 0.3, eps = 1e-6):
	inputs = K.reshape(inputs, (K.shape(inputs)[0],-1))
	targets = K.reshape(targets, (K.shape(targets)[0],-1))
	background = 1 - K.mean(targets)
	if(background<t):
		l = 1-l
	BCE = K.mean(K.binary_crossentropy(targets, inputs), axis= 1)
	intersection = K.sum(targets*inputs, axis = 1)
	Dice_Loss = 1 - (2*intersection+eps)/(K.sum(targets, axis = 1) + K.sum(inputs, axis = 1) + eps)
	Inv_inputs = 1 - inputs
	Inv_targets = 1 - targets
	Inv_intersection = K.sum(Inv_targets*Inv_inputs, axis = 1)
	Inv_Dice_Loss = 1 - (2*Inv_intersection+eps)/(K.sum(Inv_targets, axis = 1) + K.sum(Inv_inputs, axis = 1) + eps)
	Final = BCE+l*Dice_Loss + (1-l)*Inv_Dice_Loss
	return Final


In [ ]:
#DICE_COEFF
def dice_coef(y_true, y_pred, eps = 1e-6):
    y_pred = K.round(y_pred)
    intersection = K.sum(y_true*y_pred, axis = [1,2,3])
    union = K.sum(y_true, axis = [1,2,3]) + K.sum(y_pred, axis = [1,2,3])
    dice = K.mean((2*intersection + eps)/(union+eps), axis = 0)
    return dice

In [ ]:
#ARCHITECTURE GOES HERE
from keras import layers, Input
from keras import models, Model
from keras import optimizers
from keras.layers import Dense, Flatten, Activation, Dropout, MaxPooling2D, Conv2DTranspose, Conv2D, Concatenate
from keras import regularizers
from keras.layers.normalization import BatchNormalization


inputs = Input(shape=(IMG_HEIGHT,IMG_WIDTH,IMG_CHANNEL))
s = tf.keras.layers.Lambda(lambda x: x/255)(inputs)
x1 = Conv2D(16, (3,3), activation='relu', padding='same')(s)
x1 = Conv2D(16, (3,3), activation='relu', padding='same')(BatchNormalization()(x1))
x1_m = MaxPooling2D((2,2))(BatchNormalization()(x1))

x2 = Conv2D(32, (3,3), activation='relu', padding='same')(x1_m)
x2 = Conv2D(32, (3,3), activation='relu', padding='same')(BatchNormalization()(x2))
x2_m = MaxPooling2D((2,2))(BatchNormalization()(x2))

x3 = Conv2D(64, (3,3), activation='relu', padding='same')(x2_m)
x3 = Conv2D(64, (3,3), activation='relu', padding='same')(BatchNormalization()(x3))
x3_m = MaxPooling2D((2,2))(BatchNormalization()(x3))

x4 = Conv2D(128, (3,3), activation='relu', padding='same')(x3_m)
x4 = Conv2D(128, (3,3), activation='relu', padding='same')(BatchNormalization()(x4))
x4_m = MaxPooling2D((2,2))(BatchNormalization()(x4))

x5 = Conv2D(256, (3,3), activation='relu', padding='same')(x4_m)
x5 = Conv2D(256, (3,3), activation='relu', padding='same')(BatchNormalization()(x5))
x5_u = Conv2DTranspose(512, (3,3), strides=(2,2), padding='same')(BatchNormalization()(x5))

x6 = Concatenate()([x4,x5_u])
x6 = Conv2D(128, (3,3), activation='relu', padding='same')(x6)
x6 = Conv2D(128, (3,3), activation='relu', padding='same')(BatchNormalization()(x6))
x6_u = Conv2DTranspose(64, (3,3), strides=(2,2), padding='same')(BatchNormalization()(x6))

x7 = Concatenate()([x3,x6_u])
x7 = Conv2D(64, (3,3), activation='relu', padding='same')(x7)
x7 = Conv2D(64, (3,3), activation='relu', padding='same')(BatchNormalization()(x7))
x7_u = Conv2DTranspose(32, (3,3), strides=(2,2), padding='same')(BatchNormalization()(x7))

x8 = Concatenate()([x2,x7_u])
x8 = Conv2D(32, (3,3), activation='relu', padding='same')(x8)
x8 = Conv2D(32, (3,3), activation='relu', padding='same')(BatchNormalization()(x8))
x8_u = Conv2DTranspose(16, (3,3), strides=(2,2), padding='same')(BatchNormalization()(x8))

x9 = Concatenate()([x1,x8_u])
x9 = Conv2D(16, (3,3), activation='relu', padding='same')(x9)
x9 = Conv2D(16, (3,3), activation='relu', padding='same')(BatchNormalization()(x9))

# outputs = Conv2D(1, (1,1), activation='sigmoid', padding='same')(c9)
outputs = Conv2D(1, (1,1), activation='sigmoid')(BatchNormalization()(x9))
model = Model(inputs=[inputs], outputs=[outputs], name="UNET_baseline")
model.summary()

In [ ]:
def step_decay(epoch):
   initial_lrate = 0.1
   drop = 0.5
   epochs_drop = 10.0
   lrate = initial_lrate * math.pow(drop,  
           math.floor((1+epoch)/epochs_drop))
   return lrate

lrate = tf.keras.callbacks.LearningRateScheduler(step_decay)
learning_rate = 0.1
decay_rate = learning_rate / 150
momentum = 0.8
sgd = tf.keras.optimizers.SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
model.compile(loss=Switching_Loss, optimizer=optimizers.Adam(lr=1e-3), metrics=[dice_coef])

In [ ]:
from keras.callbacks  import ReduceLROnPlateau
from keras.callbacks import ModelCheckpoint
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=5)
checkpoint = ModelCheckpoint('best.h5', verbose=1, monitor='val_acc',save_best_only=True)

history = model.fit(X_gen, y_train, batch_size = 4, epochs = 200, callbacks=[checkpoint,lrate], validation_data = (X_val, y_val), shuffle= False)
# history = model.fit(Train_data, steps_per_epoch = 50, epochs = 200, callbacks=[checkpoint, learning_rate], validation_data = Val_data, validation_steps = 43)


In [ ]:
import matplotlib
import matplotlib.pyplot as plt
plt.plot(history.history['loss'][1:])
plt.plot(history.history['val_loss'][1:])
plt.legend(['Training loss', 'Validation loss'])

In [ ]:
import matplotlib.pyplot as plt
output = model.predict(X_val)
imshow(output[0,:,:,0], cmap = 'gray')

In [ ]:
import matplotlib.pyplot as plt
output = model.predict(X_train)
imshow(output[1,:,:,0], cmap = 'gray')

In [ ]:
imshow(y_val[0,:,:,0], cmap = 'gray')

In [ ]:
imshow(y_train[0,:,:,0], cmap = 'gray')